In [1]:
from pyplasm import *
import csv

Evaluating fenvs.py..
...fenvs.py imported in 0.005092 seconds


## Immagine di Riferimento

![imagine](http://systemgvp.altervista.org/images/immagini/lavori/struttura_3.jpg)

## Codice per creare il singolo reticolo

In [2]:
def createBeamsAndPillars(beamSec,pillarSec,distanceBtwPill,eightsBeam):
    maxHeight= SUM(eightsBeam)+beamSec[1]
    pillarLong=CUBOID([pillarSec[0],pillarSec[1],maxHeight]) 
    pillars=[]
    floor=[]
   
    for d in range(len(distanceBtwPill)):
        pillars.append(pillarLong)
        floor.append(T(2)(pillarSec[1]))
        
        for h in range(0,len(eightsBeam)):
            beam=CUBOID([beamSec[0],distanceBtwPill[d],beamSec[1]])
            floor.append(T(3)(eightsBeam[h]))
            floor.append(beam)
       
    
        floor.append(T(2)(pillarSec[1]))  
        img=STRUCT(floor) 
        floor=[]
        pillars.append(img)
        img=[]
        pillars.append(T(2)(distanceBtwPill[d]+pillarSec[1]))
        
    
    pillars.append(pillarLong)
    final=STRUCT(pillars)
    return final

## Metodo richiesto per l'HW

In [3]:
def ggpl_bone_structure(file_name):
    
    beamAndPillar=[]
    translation=[]
    odd=True
    with open(file_name, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            if(odd):
                x=float(row[0])
                y=float(row[1])
                z=float(row[2])
                tripla=(x,y,z)
                translation.append(tripla)
                odd=False
                
            else:
                element1= (float(row[0]),float(row[1]))
                element2= (float(row[2]),float(row[3]))
                element3=[]
                element4=[]
                cont1=int(row[4])
                for i in range(0,cont1):
                    element3.append(float(row[5+i]))
                cont2=int(row[5+cont1])
                for i in range(0,cont2):
                    element4.append(float(row[6+cont1+i]))
                
                obj=[]
                obj.append(element1)
                obj.append(element2)
                obj.append(element3)
                obj.append(element4)
                beamAndPillar.append(obj)
                
                odd=True
                      
    combineStructAndTranslation(beamAndPillar,translation)
    return 0

## Metodo che combina traslazioni e Reticoli
Raccolte le informazioni dal file csv si procede trasnlando i reticoli e concatenandoli grazie al metodo che segue

In [ ]:
def combineStructAndTranslation(beamAndPillar,translation):
    building=[]
    for i in range(0,len(translation)):
        vx=translation[i][0]
        vy=translation[i][1]
        vz=translation[i][2]
    
        building.append(T(1)(vx))
        building.append(T(2)(vy))
        building.append(T(3)(vz))
        frame=beamAndPillar[i]

        building.append(createBeamsAndPillars(frame[0],frame[1],frame[2],frame[3]))

    img=STRUCT(building)
    VIEW(img)
    return 0

## Test del 

In [ ]:
ggpl_bone_structure("framedata_461980.csv")